In [15]:
from SignalProcessing import SignalSource, SignalReader, Classifier
from SignalProcessing import SimplePulseAnalyser as analysisClass
import numpy as np
import ipywidgets as widgets
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import LinearColorMapper, ColorBar

class ProgressBar(widgets.IntProgress):
    def __init__(self, maxVal):
        return super().__init__(
            value=0,
            min=0,
            max=maxVal,
            step=1,
            description='',
            bar_style='', # 'success', 'info', 'warning', 'danger' or ''
            orientation='horizontal'
        )

output_notebook()

Loading BokehJS ...

In [6]:
loopIterations = 500

progress = ProgressBar(maxVal = loopIterations)
progress.layout.display = 'flex'
display(progress)

time = np.arange(start=0.0, stop=1000.0, step=0.1)
a = SignalSource()
b = SignalReader(noise=3e-3, numbits=12, maxVolts=0.35)

amp = np.random.uniform(low=0, high=0.4, size=loopIterations)
arrivalTime = np.random.uniform(low=0, high=1000, size=loopIterations)

cleansig = [[None]*len(time)]*loopIterations
nonsig = [[None]*len(time)]*loopIterations*2
for i in range(loopIterations):
    clean = a.cleanSignal(amplitude=amp[i], arrivalTime=arrivalTime[i])
    spark = a.spark(amplitude=amp[i], arrivalTime=arrivalTime[i])
    glitch = a.glitch(amplitude=amp[i]/10, arrivalTime=arrivalTime[i])
    
    cleansig[i] = [n for n in b.acquireData(time, clean)]
    nonsig[i*2] = [n for n in b.acquireData(time, spark)]
    nonsig[i*2 + 1] = [n for n in b.acquireData(time, glitch)]
    
    progress.value = i
    progress.description = '{} / {}'.format(i*3 + 1, loopIterations*3)
    
progress.value = progress.max
progress.bar_style = 'success'
progress.description = 'Done!'

In [18]:
sigCount = len(cleansig)
progress = ProgressBar(maxVal = sigCount + len(nonsig))
progress.layout.display = 'flex'
display(progress)

quality_clean = np.zeros(sigCount)

for sig in cleansig:
    progress.value += 1
    progress.description = '{} / {}'.format(progress.value, progress.max)
    
    analyser = analysisClass(
        t = time,
        sig = sig
    )
    try:
        quality_clean[progress.value-1] = analyser.quality()
    except RuntimeError:
        quality_clean[progress.value-1] = np.nan
    
sigCount = len(nonsig)
progress.max = sigCount

quality_nonsig = np.zeros(sigCount)

for sig in nonsig:
    progress.value += 1
    progress.description = '{} / {}'.format(progress.value, progress.max)
    
    analyser = analysisClass(
        t = time,
        sig = sig
    )
    try:
        quality_nonsig[progress.value-1] = analyser.quality()
    except RuntimeError:
        quality_nonsig[progress.value-1] = np.nan
        
progress.value = progress.max
progress.bar_style = 'success'
progress.description = 'Done!'

In [19]:
quality_clean = quality_clean[~np.isnan(quality_clean)]
quality_nonsig = quality_nonsig[~np.isnan(quality_nonsig)]

In [20]:
f = figure(width=900, height=450, tools="pan,wheel_zoom,box_zoom,reset,hover")

allData = np.concatenate((quality_clean, quality_nonsig))

bins = range(0, 550, 5)
bins = np.linspace(start=0, stop=max(allData), num=int(len(allData)/25))

histc, binsc = np.histogram(quality_clean, bins=bins)
histg, binsg = np.histogram(quality_nonsig, bins=bins)

f.quad(top=histc, bottom=0, left=binsc[:-1], right=binsc[1:], color='blue')
f.quad(top=histg+histc, bottom=histc, left=binsg[:-1], right=binsg[1:], color='red')

show(f)

In [21]:
progress = ProgressBar(maxVal = 150)
progress.layout.display = 'flex'
display(progress)

MCC = np.zeros((150, 250)) * np.nan
for cutlow in range(150):
    progress.value = cutlow
    progress.description = '{} / 150'.format(cutlow)
    for cuthigh in range(cutlow+2, 250):
        cl = Classifier(events=quality_clean, nonevents=quality_nonsig, cuts=[cutlow, cuthigh])
        MCC[cutlow, cuthigh] = cl.MCC

progress.description = 'Done!'
progress.bar_style = 'success'
color_mapper = LinearColorMapper(palette="Plasma256", low=-1.0, high=1.0)

p = figure(width=900, height=450, x_range=(0, 250), y_range=(0, 150))
p.image(image=[MCC], x=0, y=0, dw=250, dh=150, color_mapper=color_mapper)

color_bar = ColorBar(color_mapper=color_mapper, location=(0,0))

p.add_layout(color_bar, 'left')

show(p)

In [22]:
cuts = np.unravel_index(np.nanargmax(MCC), MCC.shape)
cl = Classifier(events=quality_clean, nonevents=quality_nonsig, cuts=cuts)

print('cuts = ', cuts)
print(' ')
print('TP = ', 100*cl.TP/cl.P, '%')
print('TN = ', 100*cl.TN/cl.N, '%')
print('FP = ', 100*cl.FP/cl.P, '%')
print('FN = ', 100*cl.FN/cl.N, '%')
print(' ')
print('MCC = ', cl.MCC)

cuts =  (35, 98)
 
TP =  89.0438247012 %
TN =  94.6893787575 %
FP =  10.9561752988 %
FN =  5.31062124248 %
 
MCC =  0.838165602383
